In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier


In [2]:

# Membaca data dari file CSV
df = pd.read_csv('gldm_features_Data_Normal.csv')

# Membagi data menjadi beberapa dataframe berdasarkan arah
directions = ['0', '45', '90', '135']
dataframes = []

for direction in directions:
    cols = [f'Contrast_{direction}', f'ASM_{direction}', f'Entropy_{direction}', f'Mean_{direction}',
            f'IDM_{direction}']
    df_direction = df[['Image'] + cols]
    df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
    dataframes.append(df_direction)

C:\Users\hanif\AppData\Local\Temp\ipykernel_1144\2209788452.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_1144\2209788452.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_direction['Label'] = df.index.map(lambda x: 0 if x <= 600 else 1)
C:\Users\hanif\AppData\Local\Temp\ipykernel_1144\2209788452.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [3]:
# Fungsi untuk melatih dan menguji model Random Forest
def train_test_RF(X_train, y_train, X_test, y_test):
    scaler = PowerTransformer()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Daftar kombinasi hiperparameter yang akan diuji
    param_grid = {'n_estimators': [50, 100, 150],
                  'max_depth': [None, 5, 10]}
    
    rf = RandomForestClassifier()

    # Mencari kombinasi hiperparameter terbaik menggunakan Grid Search
    grid_search = GridSearchCV(rf, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    # Menggunakan model terbaik untuk prediksi
    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, best_rf



# List untuk menyimpan akurasi per parameter dan direction
accuracies = []

# Daftar parameter yang akan diuji
parameters = ['Contrast', 'ASM', 'Entropy', 'Mean', 'IDM']

# Melakukan train dan test untuk setiap parameter dan direction
for parameter in parameters:
    for i, df_direction in enumerate(dataframes):
        # Memilih kolom dataframe sesuai parameter
        cols = [col for col in df_direction.columns if parameter in col]
        df_parameter_direction = df_direction[['Image', 'Label'] + cols]
        
        X = df_parameter_direction.drop(['Image', 'Label'], axis=1)
        y = df_parameter_direction['Label']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        accuracy, best_svm = train_test_RF(X_train, y_train, X_test, y_test)
        accuracies.append(accuracy)
        
        print(f"Akurasi {parameter} Direction {directions[i]}")
        #print(df_parameter_direction)
        
        print(f"Accuracy: {accuracy}")
        print("Best SVM Model:", best_svm)
        print()

Akurasi Contrast Direction 0
Accuracy: 0.5
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=50)

Akurasi Contrast Direction 45
Accuracy: 0.55
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=50)

Akurasi Contrast Direction 90
Accuracy: 0.525
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=150)

Akurasi Contrast Direction 135
Accuracy: 0.49166666666666664
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=150)

Akurasi ASM Direction 0
Accuracy: 0.525
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=50)

Akurasi ASM Direction 45
Accuracy: 0.5166666666666667
Best SVM Model: RandomForestClassifier(max_depth=5)

Akurasi ASM Direction 90
Accuracy: 0.5125
Best SVM Model: RandomForestClassifier(max_depth=5)

Akurasi ASM Direction 135
Accuracy: 0.5
Best SVM Model: RandomForestClassifier(max_depth=5, n_estimators=50)

Akurasi Entropy Direction 0
Accuracy: 0.6666666666666666
Best SVM Model: RandomForestClassifier(max_d

In [4]:
# Membuat DataFrame kosong
results_df = pd.DataFrame(columns=['Parameter', '0', '45', '90', '135', 'Average'])

# Daftar parameter yang akan diuji
parameters = ['Contrast', 'ASM', 'Entropy', 'Mean', 'IDM']

# Melakukan iterasi untuk setiap parameter
for i, parameter in enumerate(parameters):
    # Menambahkan baris baru ke DataFrame
    results_df.loc[i] = [parameter] + accuracies[i*4:(i+1)*4] + [sum(accuracies[i*4:(i+1)*4])/4]

# Menampilkan DataFrame hasil
print(results_df)

  Parameter         0        45        90       135   Average
0  Contrast  0.500000  0.550000  0.525000  0.491667  0.516667
1       ASM  0.525000  0.516667  0.512500  0.500000  0.513542
2   Entropy  0.666667  0.670833  0.658333  0.662500  0.664583
3      Mean  0.541667  0.533333  0.554167  0.487500  0.529167
4       IDM  0.529167  0.525000  0.541667  0.525000  0.530208
